In [19]:
import pdfminer
from PyPDF2 import PdfFileReader
import nltk
import numpy as np
import re
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [20]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter#process_pdf
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams

from cStringIO import StringIO

def pdf_to_text(pdfname):

    # PDFMiner boilerplate
    rsrcmgr = PDFResourceManager()
    sio = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, sio, codec=codec, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    # Extract text
    fp = file(pdfname, 'rb')
    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
    fp.close()

    # Get text from StringIO
    text = sio.getvalue()

    # Cleanup
    device.close()
    sio.close()

    return text

In [21]:
def get_specific_section(text, section_name):
    
    end_section = ''
    lines = list(filter(bool,text.split('\n')))
    for i in range(len(lines)):
        start_section = '[0-9]+.[0-9]+' + '(.*)  '+  'Quality aspects'
        if re.match(start_section, lines[i]):
            start_section_number = lines[i].split()[0]
            start_section_head = re.findall('[0-9].', start_section_number)[0]
            start_section_name = lines[i]
            end_section_number = start_section_head + str(int(start_section_number.strip('.')[-1]) + 1)
            end = end_section_number + '(.*)'+'  [A-Za-z]*'
    end_section += end
    end_section_name = []
    for i in range(len(lines)):
        if re.match(end_section, lines[i]):
            end_section_name.append(lines[i])

    str1 = text[text.find(start_section_name)+len(start_section_name):text.rfind(end_section_name[0])]
    return str1

In [22]:
CLUSTER_THRESHOLD = 5
TOP_SENTENCES = 10

In [23]:
def _score_sentences(sentences, important_words):
    scores = []
    sentence_idx = -1
    
    for s in [nltk.tokenize.word_tokenize(s) for s in sentences]:
        sentence_idx += 1
        word_idx = []
        
        #For each word in the word list...
        for w in important_words:
            try:
                #Compute an index for where any important word occurs in the sentence.
                word_idx.append(s.index(w))
            except ValueError as e: # w not in this particular sentence
                pass
            
        word_idx.sort()
        
        #It is possible that some sentences may not contain any important words at all.
        if len(word_idx) == 0: continue
            
        #Using the word index, compute clusters by using a max distance threshold for any two consecutive words
        
        clusters = []
        cluster = [word_idx[0]]
        i = 1
        while i < len(word_idx):
            if word_idx[i] - word_idx[i-1] < CLUSTER_THRESHOLD:
                cluster.append(word_idx[i])
            else:
                clusters.append(cluster[:])
                cluster = [word_idx[i]]
            i += 1
        clusters.append(cluster)
        
        #Scores for each cluster. The max score for any given cluster is the score for the sentence.
        
        max_cluster_score = 0
        for c in clusters:
            significant_words_in_cluster = len(c)
            total_words_in_cluster = c[-1] - c[0] + 1
            score = 1.0 * significant_words_in_cluster \
                    * significant_words_in_cluster / total_words_in_cluster
            
            if score > max_cluster_score:
                max_cluster_score = score
                
        scores.append((sentence_idx, score))
    return scores

In [24]:
def summarize(txt):
    txt = txt.replace("\n","")
    sentences = [s for s in nltk.tokenize.sent_tokenize(txt)]
    normalized_sentences = [s.lower() for s in sentences]
    words = [w.lower() for sentence in normalized_sentences for w in nltk.tokenize.word_tokenize(sentence)]
    fdist = nltk.FreqDist(words)
    top_n_words = [w[0] for w in fdist.items()
                  if w[0] not in nltk.corpus.stopwords.words('english')][:]
    scored_sentences = _score_sentences(normalized_sentences, top_n_words)

    #Summarization Approach 1:
    #Filter out nonsignificant sentences by using the average scores plus a fraction of the std dev as a filter

    avg = np.mean([s[1] for s in scored_sentences])
    std = np.std([s[1] for s in scored_sentences])
    mean_scored = [(sent_idx, score) for (sent_idx, score) in scored_sentences 
                   if score > avg + 0.5 * std]

    #Summarization Approach 2:
    #Another approach would be to return only the top N ranked sentences

    top_n_scored = sorted(scored_sentences, key=lambda s: s[1])[-TOP_SENTENCES:]
    top_n_scored = sorted(top_n_scored, key=lambda s:s[0])
    
    return dict(top_n_summary = [sentences[idx] for (idx, score) in top_n_scored], 
               mean_scored_summary = [sentences[idx] for (idx, score) in mean_scored])




In [25]:
text1 = pdf_to_text("/Users/itziar/Downloads/DS_NLP_Assignment_final/WC500057122.pdf")
text2 = pdf_to_text("/Users/itziar/Downloads/DS_NLP_Assignment_final/WC500135744.pdf")

section1 = get_specific_section(text1, 'Non-clinical aspects')
section2 = get_specific_section(text2, 'Non-clinical aspects')

In [32]:
#summarize(section1)

In [33]:
#summarize(section2)